In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/3.model_code.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Vehicle Type Code'] = input_['Vehicle Type Code'].astype(int)
input_['Make Code'] = input_['Make Code'].astype(int)
input_['Model Code'] = input_['Model Code'].astype(int)
input_['Year Code'] = input_['Year Code'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1

                try:
                    resp = requests.get(f'''https://www.moogparts.com/driv/partfinder/api.catalog.partslist?brand=moog&locale=en_US&country_code=US&year_id={input_.loc[a, 'Year Code']}&make_id={input_.loc[a, 'Make Code']}&model_id={input_.loc[a, 'Model Code']}&vehicle_group_ids={input_.loc[a, 'Vehicle Type Code']}''',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))
                    
                    if resp.status_code == 200:
                        json_ = resp.json()

                        list_part = json_['application_list']['applications']
                        if list_part:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            list_brand = [part['brand'] for part in list_part]

            # = = = = = = = = = = = = = = =

            list_part_number = [part['part_number'] for part in list_part]
            
            # = = = = = = = = = = = = = = =

            list_url = [part['part_detail_redirect_url'].strip() for part in list_part]

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'No': '',
                                    'Brand': list_brand,
                                    'Part Number': list_part_number,
                                    'Url': list_url})
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
            
            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Vehicle Type Code']} > {input_.loc[a, 'Make Code']} > {input_.loc[a, 'Model Code']} > {input_.loc[a, 'Year Code']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Brand', 'Part Number', 'Url'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct = output_correct.drop_duplicates(['Url'],
                                                    keep='first',
                                                    ignore_index=True)
    output_correct['No'] = [i+1 for i in range(len(output_correct))]
    output_correct.to_excel(f'''./file/4.part_number-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Vehicle Type Code', 'Make Code', 'Model Code', 'Year Code'],
                                            ascending=[True, True, True, True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/part_number_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：97] - 8 > 54 > 26368 > 1983
[剩余数量：1000] - [当前时间：18:30:44]

[状态：ok，尝试次数：9] - 8 > 71 > 22652 > 1993
[剩余数量：999] - [当前时间：18:30:44]

[状态：ok，尝试次数：106] - 8 > 54 > 25225 > 1995
[剩余数量：998] - [当前时间：18:30:44]

[状态：ok，尝试次数：91] - 8 > 54 > 26368 > 1984
[剩余数量：997] - [当前时间：18:30:46]

[状态：ok，尝试次数：211] - 8 > 54 > 24941 > 1987
[剩余数量：996] - [当前时间：18:30:46]

[状态：ok，尝试次数：10] - 8 > 71 > 22652 > 1994
[剩余数量：995] - [当前时间：18:30:47]

[状态：ok，尝试次数：3] - 8 > 71 > 22716 > 1987
[剩余数量：994] - [当前时间：18:30:47]

[状态：ok，尝试次数：21] - 8 > 71 > 22652 > 1996
[剩余数量：993] - [当前时间：18:30:47]

[状态：ok，尝试次数：11] - 8 > 71 > 22716 > 1990
[剩余数量：992] - [当前时间：18:30:48]

[状态：ok，尝试次数：17] - 8 > 71 > 22652 > 1998
[剩余数量：991] - [当前时间：18:30:49]

[状态：ok，尝试次数：56] - 8 > 71 > 22652 > 1989
[剩余数量：990] - [当前时间：18:30:50]

[状态：ok，尝试次数：24] - 8 > 71 > 22652 > 2000
[剩余数量：989] - [当前时间：18:30:50]

[状态：ok，尝试次数：5] - 8 > 71 > 22721 > 1979
[剩余数量：988] - [当前时间：18:30:51]

[状态：ok，尝试次数：9] - 8 > 71 > 22721 > 1981
[剩余数量：987] - [当前时间：18:30:51]

[状态：ok，尝试次数：53] - 8 >